In [ ]:
# common imports
import os
import numpy as np
from tqdm import tqdm
import random
import time
import warnings
warnings.filterwarnings("ignore")
from pathlib import Path
from tempfile import gettempdir
from typing import Dict
import pandas as pd
import matplotlib.pyplot as plt

# torch imports
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.models.resnet import resnet50, resnet18, resnet34, resnet101
import torch.nn.functional as F


# l5kit imports
import l5kit
from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

In [ ]:
l5kit.__version__

In [ ]:
torch.cuda.is_available()

In [ ]:
def find_no_of_trainable_params(model):
    total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    #print(total_trainable_params)
    return total_trainable_params

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
set_seed(42)

In [ ]:
#!rm /kaggle/working/*

## Configs

In [ ]:
# --- Lyft configs ---
cfg = {
    'format_version': 4,
    'data_path': "../input/lyft-motion-prediction-autonomous-vehicles/",
    'model_params': {
        'model_architecture': 'resnet34',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
        'model_name': "R34_pvt_10_224_norm_agent_features_nll_multimode_ReducedLR",
        'lr': 7e-4,
        'weight_path': "../input/lyft-motion-prediction-resnet-weight-files/R34_pvt_10_224_norm_agent_features_nll_multimode_ReduceLr_1752k.pth",
        'train': True,
        'predict': False
    },

    'raster_params': {
        'raster_size': [224, 224],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },

    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 32,
        'shuffle': True,
        'num_workers': 4
    },
    
    'val_data_loader': {
        'key': 'scenes/validate.zarr',
        'batch_size': 16,
        'shuffle': True,
        'num_workers': 4
    },
    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 32,
        'shuffle': False,
        'num_workers': 4
    },

    'train_params': {
        'train_start_index' : 54751,
        'max_num_steps': 8000,
        'checkpoint_every_n_steps': 250,
    }
}

In [ ]:
NUMBER_OF_HISTORY_FRAMES = cfg['model_params']['history_num_frames'] + 1
RASTER_IMG_SIZE = cfg['raster_params']['raster_size'][0]
NUM_MODES = 3
NUMBER_OF_FUTURE_FRAMES = cfg['model_params']['future_num_frames']
TRAIN_BATCH_SIZE = cfg['train_data_loader']['batch_size'] 
### TRAIN FROM WHERE LEFT OFF, CHANGE THE STARTING INDICES VARIABLE ACCORDINGLY
TRAIN_START_INDICES = cfg['train_params']['train_start_index']
EXTENT_RANGE = 5.0 

## Load the train and test data

In [ ]:
# set env variable for data
DIR_INPUT = cfg["data_path"]
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)
rasterizer = build_rasterizer(cfg, dm)

In [ ]:
# ===== INIT TRAIN DATASET============================================================
train_cfg = cfg["train_data_loader"]
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)

In [ ]:
print('Length of Train dataset is ' ,len(train_dataset))
print("==================================TRAIN DATA==================================")
print(train_dataset)

In [ ]:
sampled_indices = np.random.choice(len(train_dataset), size = len(train_dataset), replace = False)
print('Before slicing, start indices are ', sampled_indices[0:10])

In [ ]:
TRAIN_START_INDICES

In [ ]:
sampled_indices = sampled_indices[TRAIN_START_INDICES:]
print('After slicing, start indices are ', sampled_indices[0:10])

In [ ]:
Datasampler = SubsetRandomSampler(sampled_indices)

In [ ]:
train_dataloader = DataLoader(train_dataset, sampler=Datasampler, batch_size=train_cfg["batch_size"], 
                             num_workers=train_cfg["num_workers"])

## Loss function

In [ ]:
# --- Function utils ---
# Original code from https://github.com/lyft/l5kit/blob/20ab033c01610d711c3d36e1963ecec86e8b85b6/l5kit/l5kit/evaluation/metrics.py
from torch import Tensor


def pytorch_neg_multi_log_likelihood_batch(
    gt: Tensor, pred: Tensor, confidences: Tensor, avails: Tensor
) -> Tensor:
    """
    Compute a negative log-likelihood for the multi-modal scenario.
    log-sum-exp trick is used here to avoid underflow and overflow, For more information about it see:
    https://en.wikipedia.org/wiki/LogSumExp#log-sum-exp_trick_for_log-domain_calculations
    https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    https://leimao.github.io/blog/LogSumExp/
    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(modes)x(time)x(2D coords)
        confidences (Tensor): array of shape (bs)x(modes) with a confidence for each mode in each sample
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    assert len(pred.shape) == 4, f"expected 3D (MxTxC) array for pred, got {pred.shape}"
    batch_size, num_modes, future_len, num_coords = pred.shape

    assert gt.shape == (batch_size, future_len, num_coords), f"expected 2D (Time x Coords) array for gt, got {gt.shape}"
    assert confidences.shape == (batch_size, num_modes), f"expected 1D (Modes) array for gt, got {confidences.shape}"
    assert torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))), "confidences should sum to 1"
    assert avails.shape == (batch_size, future_len), f"expected 1D (Time) array for gt, got {avails.shape}"
    # assert all data are valid
    assert torch.isfinite(pred).all(), "invalid value found in pred"
    assert torch.isfinite(gt).all(), "invalid value found in gt"
    assert torch.isfinite(confidences).all(), "invalid value found in confidences"
    assert torch.isfinite(avails).all(), "invalid value found in avails"

    # convert to (batch_size, num_modes, future_len, num_coords)
    gt = torch.unsqueeze(gt, 1)  # add modes
    avails = avails[:, None, :, None]  # add modes and cords

    # error (batch_size, num_modes, future_len)
    error = torch.sum(((gt - pred) * avails) ** 2, dim=-1)  # reduce coords and use availability

    with np.errstate(divide="ignore"):  # when confidence is 0 log goes to -inf, but we're fine with it
        # error (batch_size, num_modes)
        error = torch.log(confidences) - 0.5 * torch.sum(error, dim=-1)  # reduce time

    # use max aggregator on modes for numerical stability
    # error (batch_size, num_modes)
    max_value, _ = error.max(dim=1, keepdim=True)  # error are negative at this point, so max() gives the minimum one
    error = -torch.log(torch.sum(torch.exp(error - max_value), dim=-1, keepdim=True)) - max_value  # reduce modes
    # print("error", error)
    return torch.mean(error)


def pytorch_neg_multi_log_likelihood_single(
    gt: Tensor, pred: Tensor, avails: Tensor
) -> Tensor:
    """

    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(time)x(2D coords)
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    # pred (bs)x(time)x(2D coords) --> (bs)x(mode=1)x(time)x(2D coords)
    # create confidence (bs)x(mode=1)
    batch_size, future_len, num_coords = pred.shape
    confidences = pred.new_ones((batch_size, 1))
    return pytorch_neg_multi_log_likelihood_batch(gt, pred.unsqueeze(1), confidences, avails)

## Model

Next we define the baseline model. Note that this model will return three possible trajectories together with confidence score for each trajectory.

In [ ]:
class LyftMultiModel(nn.Module):

    def __init__(self, cfg: Dict, num_modes=3):
        super().__init__()

        architecture = cfg["model_params"]["model_architecture"]
        backbone = eval(architecture)(pretrained=True)
        self.backbone = backbone

        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels

        self.backbone.conv1 = nn.Conv2d(
            num_in_channels,
            self.backbone.conv1.out_channels,
            kernel_size=self.backbone.conv1.kernel_size,
            stride=self.backbone.conv1.stride,
            padding=self.backbone.conv1.padding,
            bias=False,
        )

        # This is 512 for resnet18 and resnet34;
        # And it is 2048 for the other resnets
        
        if architecture == "resnet50":
            backbone_out_features = 2048
        else:
            backbone_out_features = 512

        self.dropout = nn.Dropout(p=0.3)
            
        # X, Y coords for the future positions (output shape: batch_sizex50x2)
        self.future_len = cfg["model_params"]["future_num_frames"]
        num_targets = 2 * self.future_len
        other_agent_features = num_history_channels + 3 # extent info is 3d 
        total_num_features = backbone_out_features + other_agent_features
        self.head = nn.Linear(in_features=total_num_features, out_features=1024)

        # final prediction - a fc layer with desired number of outputs, no activation
        self.num_preds = num_targets * num_modes
        self.num_modes = num_modes
        self.logit = nn.Linear(1024, out_features=self.num_preds + num_modes)

    def forward(self, x, agent_data):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)
        
        # adding other agent data to image data
        x = torch.cat((x, agent_data), dim=1)

        # fc with relu activation and dropout
        x = self.dropout(F.relu(self.head(x)))
        x = self.logit(x)

        # pred (batch_size)x(modes)x(time)x(2D coords)
        # confidences (batch_size)x(modes)
        bs, _ = x.shape
        pred, confidences = torch.split(x, self.num_preds, dim=1)
        pred = pred.view(bs, self.num_modes, self.future_len, 2)
        assert confidences.shape == (bs, self.num_modes)
        confidences = torch.softmax(confidences, dim=1)
        return pred, confidences

In [ ]:
def forward(data, model, device, criterion = pytorch_neg_multi_log_likelihood_batch):
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].to(device)
    targets = data["target_positions"].to(device)
    bs = inputs.shape[0]
    history_positions = data['history_positions'].to(device).view(bs, -1)
    # centroid = data['centroid'].to(device).float()
    # yaw = data['yaw'].to(device).view(TRAIN_BATCH_SIZE, 1).float()
    # agent_data = torch.cat((history_positions, centroid, yaw, extent), dim=1)
    extent = data['extent'].to(device) / EXTENT_RANGE
    agent_data = torch.cat((history_positions, extent), dim=1)
    
    # Forward pass
    preds, confidences = model(inputs, agent_data)
    loss = criterion(targets, preds, confidences, target_availabilities)
    return loss, preds, confidences

In [ ]:
# ==== INIT MODEL=================
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device {device}')

## Model, optimizer, Learning rate schedulers

In [ ]:
## Learning rate scheudler params
## https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.ReduceLROnPlateau
red_factor = 0.8
patience_steps = 1000 
patience_threshold = 0.1 
smoothing = 0.02

In [ ]:
model = LyftMultiModel(cfg)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=cfg["model_params"]["lr"])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=red_factor, 
                                                 patience=patience_steps, threshold= patience_threshold,
                                                verbose= True)

In [ ]:
optimizer.state_dict()["param_groups"][0]["lr"]

In [ ]:
# load weight if there is a pretrained model
weight_path = cfg["model_params"]["weight_path"]
if weight_path != '':
    checkpoint = torch.load(weight_path)

In [ ]:
model.load_state_dict(checkpoint['state_dict'])

In [ ]:
optimizer.load_state_dict(checkpoint['optimizer'])
for param_group in optimizer.param_groups:
    param_group['lr'] = cfg["model_params"]["lr"] 

In [ ]:
optimizer.state_dict()["param_groups"][0]["lr"]

In [ ]:
find_no_of_trainable_params(model)

## Training loop

Next let us implement the training loop, when the **train** parameter is set to True. 

In [ ]:
TRAIN_START_INDICES

In [ ]:
print('TRAINING ABOUT TO START ... FROM ', TRAIN_START_INDICES, 
      '  BATCH AND FOR ', cfg['train_params']['max_num_steps'], ' BATCHES', ' WITH BATCH SIZE', TRAIN_BATCH_SIZE)

In [ ]:
# ==== TRAINING LOOP =========================================================
if cfg["model_params"]["train"]:
    
    tr_it = iter(train_dataloader)
    progress_bar = tqdm(range(TRAIN_START_INDICES, 
                              TRAIN_START_INDICES + cfg["train_params"]["max_num_steps"]))
    num_iter = cfg["train_params"]["max_num_steps"]
    losses_train = []
    smooth_losses = []
    lr_list = []
    iterations = []
    metrics = []
    times = []
    model_name = cfg["model_params"]["model_name"]
    start = time.time()
    iteration = 0
    
    for i in progress_bar:
        try:
            data = next(tr_it)
        except StopIteration:
            tr_it = iter(train_dataloader)
            data = next(tr_it)
        
        # Forward pass
        model.train()
        torch.set_grad_enabled(True)
        loss, _, _ = forward(data, model, device)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses_train.append(loss.item())

        progress_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train)}")
        if i % cfg['train_params']['checkpoint_every_n_steps'] == 0:
            sample_number = i * cfg['train_data_loader']['batch_size']            
            state = {
              'state_dict': model.state_dict(),
              'optimizer': optimizer.state_dict(),
              'scheduler' : scheduler
            }
            torch.save(state, f'{model_name}_{sample_number}k.pth')
            iterations.append(i)
            metrics.append(np.mean(losses_train))
            times.append((time.time()-start)/60)
            
        # smooth the loss
        if i== TRAIN_START_INDICES:
            smooth_losses.append(loss.item())
        else:
            smooth_losses.append(smoothing  * loss.item() + (1 - smoothing) * smooth_losses[-1])
    
        scheduler.step(smooth_losses[-1])
        lr_list.append(optimizer.state_dict()["param_groups"][0]["lr"])

    sample_number = i * cfg['train_data_loader']['batch_size']
    results = pd.DataFrame({'iterations': iterations, 'metrics (avg)': metrics, 'elapsed_time (mins)': times})
    results.to_csv(f"train_metrics_{model_name}_{sample_number}k.csv", index = False)
    train_losses_csv = pd.DataFrame({'iteration': TRAIN_START_INDICES + np.arange(len(losses_train)), 
                                 'losses_train': losses_train,
                                 'smooth_losses': smooth_losses, 
                                    'learning_rate' : lr_list})
    train_losses_csv.to_csv(f"train_losses_{model_name}_{sample_number}k.csv", index = False)
    print(f"Total training time is {(time.time()-start)/60} mins")
    print(results.head())

In [ ]:
def plot_raw_vs_smooth_loss(raw_loss, smooth_losses, start_idx =0, end_idx = None):
    if end_idx is None:
        end_idx = len(raw_loss)
    plt.figure(figsize=(8, 8))
    plt.plot(range(start_idx,end_idx), raw_loss[start_idx:end_idx], label = 'raw_loss');
    plt.plot(range(start_idx,end_idx), smooth_losses[start_idx:end_idx], label = 'smoothened loss');
    plt.xlabel('Training_steps')
    plt.ylabel('Train_losses')
    plt.grid(True)
    plt.legend()

In [ ]:
plot_raw_vs_smooth_loss(losses_train, smooth_losses)

## Evaluation

In [ ]:
def model_validation_score(model, pred_path):
    # ==== EVAL LOOP
    model.eval()
    torch.set_grad_enabled(False)

    # store information for evaluation
    future_coords_offsets_pd = []
    timestamps = []
    confidences_list = []
    agent_ids = []
    progress_bar = tqdm(eval_dataloader)

    for data in progress_bar:

        _, preds, confidences = forward(data, model, device)

        #fix for the new environment
        preds = preds.cpu().numpy()
        world_from_agents = data["world_from_agent"].numpy()
        centroids = data["centroid"].numpy()
        coords_offset = []

        # convert into world coordinates and compute offsets
        for idx in range(len(preds)):
            for mode in range(3):
                preds[idx, mode, :, :] = transform_points(preds[idx, mode, :, :], world_from_agents[idx]) - centroids[idx][:2]

        future_coords_offsets_pd.append(preds.copy())
        confidences_list.append(confidences.cpu().numpy().copy())
        timestamps.append(data["timestamp"].numpy().copy())
        agent_ids.append(data["track_id"].numpy().copy())  
    
    write_pred_csv(pred_path,
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd),
               confs=np.concatenate(confidences_list),
              )
    
    metrics = compute_metrics_csv(eval_gt_path, pred_path, [neg_multi_log_likelihood, time_displace])
    for metric_name, metric_mean in metrics.items():
        print(metric_name, metric_mean)
        np.save(metric_name + '.npy', metric_mean)
        np.savetxt(metric_name + '.csv', metric_mean, delimiter=',')
    #return [future_coords_offsets_pd, confidences_list, timestamps, agent_ids]

In [ ]:
run_validation = False

if run_validation == True:
    eval_base_path = "../input/lyft-validation-chopped-100/validate_chopped_100"
    eval_cfg = cfg["val_data_loader"]
    eval_zarr_path = str(Path(eval_base_path) / Path(dm.require(eval_cfg["key"])).name)
    eval_mask_path = str(Path(eval_base_path) / "mask.npz")
    eval_gt_path = str(Path(eval_base_path) / "gt.csv")

    eval_zarr = ChunkedDataset(eval_zarr_path).open()
    eval_mask = np.load(eval_mask_path)["arr_0"]
    # ===== INIT DATASET AND LOAD MASK
    eval_dataset = AgentDataset(cfg, eval_zarr, rasterizer, agents_mask=eval_mask)
    eval_dataloader = DataLoader(eval_dataset, shuffle=eval_cfg["shuffle"], batch_size=eval_cfg["batch_size"], 
                                 num_workers=eval_cfg["num_workers"])
    print(eval_dataset)
    
    pred_path = f"{gettempdir()}/pred.csv"
    model_validation_score(model, pred_path)
    